In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install surprise

In [ ]:
import pandas as pd
import  numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from surprise import Dataset, Reader, KNNWithMeans,SVD, SVDpp,NMF, BaselineOnly, accuracy
from surprise.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.metrics import mean_squared_error
import time
from collections import defaultdict

ModuleNotFoundError: No module named 'surprise'

In [ ]:
movies        = pd.read_csv('/content/drive/MyDrive/KU Module/Project Dissertation/Project_Model/movies.csv')
links         = pd.read_csv('/content/drive/MyDrive/KU Module/Project Dissertation/Project_Model/links.csv')
ratings       = pd.read_csv('/content/drive/MyDrive/KU Module/Project Dissertation/Project_Model/ratings.csv')
tags          = pd.read_csv('/content/drive/MyDrive/KU Module/Project Dissertation/Project_Model/tags.csv')

In [ ]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [ ]:
movies.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

In [ ]:
print(movies['genres'].nunique())
print(movies['genres'].unique())

951
['Adventure|Animation|Children|Comedy|Fantasy'
 'Adventure|Children|Fantasy' 'Comedy|Romance' 'Comedy|Drama|Romance'
 'Comedy' 'Action|Crime|Thriller' 'Adventure|Children' 'Action'
 'Action|Adventure|Thriller' 'Comedy|Horror'
 'Adventure|Animation|Children' 'Drama' 'Action|Adventure|Romance'
 'Crime|Drama' 'Drama|Romance' 'Action|Comedy|Crime|Drama|Thriller'
 'Comedy|Crime|Thriller' 'Crime|Drama|Horror|Mystery|Thriller'
 'Drama|Sci-Fi' 'Children|Drama' 'Adventure|Drama|Fantasy|Mystery|Sci-Fi'
 'Mystery|Sci-Fi|Thriller' 'Children|Comedy' 'Drama|War'
 'Action|Crime|Drama' 'Action|Adventure|Fantasy' 'Comedy|Drama|Thriller'
 'Mystery|Thriller' 'Animation|Children|Drama|Musical|Romance'
 'Crime|Mystery|Thriller' 'Adventure|Drama' 'Drama|Thriller'
 'Comedy|Crime' 'Action|Sci-Fi|Thriller' 'Action|Comedy|Horror|Thriller'
 'Comedy|Drama' 'Documentary' 'Action|Crime|Drama|Thriller'
 'Crime|Drama|Romance' 'Action|Adventure|Drama' 'Action|Thriller'
 'Drama|Horror|Thriller' 'Comedy|Horror|Roman

In [ ]:
links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [ ]:
links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9742 non-null   int64  
 1   imdbId   9742 non-null   int64  
 2   tmdbId   9734 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 228.5 KB


In [ ]:
links.isnull().sum() # There is some missing values in links dataframe

movieId    0
imdbId     0
tmdbId     8
dtype: int64

In [ ]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [ ]:
ratings.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [ ]:
ratings['userId'].nunique()

610

In [ ]:
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [ ]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userId     3683 non-null   int64 
 1   movieId    3683 non-null   int64 
 2   tag        3683 non-null   object
 3   timestamp  3683 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


In [ ]:
tags.isnull().sum()

userId       0
movieId      0
tag          0
timestamp    0
dtype: int64

In [ ]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [ ]:
ratings['rating'].unique()

array([4. , 5. , 3. , 2. , 1. , 4.5, 3.5, 2.5, 0.5, 1.5])

# Movie Recommendation : KNNWithMeans

In [ ]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.20)

In [ ]:
# Initialize and train the KNNWithMeans model
model = KNNWithMeans(sim_options={'name': 'cosine', 'user_based': True})
start_time = time.time()
model.fit(trainset)
fit_time = time.time() - start_time
print(f"KNNWithMeans training completed in {fit_time:.2f} seconds.")

Computing the cosine similarity matrix...
Done computing similarity matrix.
KNNWithMeans training completed in 0.16 seconds.


In [ ]:
# Make predictions on the test set
predictions = model.test(testset)
predictions

[Prediction(uid=352, iid=53953, r_ui=4.0, est=3.4591165460998194, details={'actual_k': 20, 'was_impossible': False}),
 Prediction(uid=89, iid=48649, r_ui=4.5, est=3.5028821837655575, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=68, iid=38061, r_ui=3.5, est=3.9021743382868066, details={'actual_k': 29, 'was_impossible': False}),
 Prediction(uid=140, iid=6386, r_ui=4.0, est=3.5028821837655575, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'}),
 Prediction(uid=367, iid=899, r_ui=5.0, est=4.423711824465274, details={'actual_k': 34, 'was_impossible': False}),
 Prediction(uid=263, iid=163, r_ui=4.0, est=3.8290406716526464, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=254, iid=3510, r_ui=2.5, est=3.990918156807587, details={'actual_k': 27, 'was_impossible': False}),
 Prediction(uid=91, iid=2122, r_ui=4.0, est=2.658186348772478, details={'actual_k': 6, 'was_impossible': False}),
 Prediction(uid=68, 

In [ ]:
predictions[0]

Prediction(uid=352, iid=53953, r_ui=4.0, est=3.4591165460998194, details={'actual_k': 20, 'was_impossible': False})

In [ ]:
# Function to calculate precision and recall at k
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()

    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold)) for (est, true_r) in user_ratings[:k])

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return sum(prec for prec in precisions.values()) / len(precisions), sum(rec for rec in recalls.values()) / len(recalls)

# Evaluate model performance
precision, recall = precision_recall_at_k(predictions)
mse = accuracy.mse(predictions, verbose=False)
print(f"KNNWithMeans: Precision={precision:.3f}, Recall={recall:.3f}, MSE={mse:.3f}, RMSE={np.sqrt(mse):.3f}")

KNNWithMeans: Precision=0.709, Recall=0.511, MSE=0.829, RMSE=0.910


In [ ]:
# Function to generate recommendations
def generate_recommendations(user_id, predictions):
    user_predictions = [pred for pred in predictions if pred.uid == user_id]
    user_predictions.sort(key=lambda x: x.est, reverse=True)
    top_10_recommendations = user_predictions[:10]

    print(f"Some Movies recommendations for User {user_id}:")
    for pred in top_10_recommendations:
        print(f"Movie ID: {pred.iid}, Estimated Rating: {pred.est:.2f}")

# Example usage
user_id = 1
generate_recommendations(user_id, predictions)

Some Movies recommendations for User 1:
Movie ID: 940, Estimated Rating: 4.88
Movie ID: 1222, Estimated Rating: 4.82
Movie ID: 2542, Estimated Rating: 4.79
Movie ID: 260, Estimated Rating: 4.74
Movie ID: 1206, Estimated Rating: 4.74
Movie ID: 2761, Estimated Rating: 4.69
Movie ID: 1220, Estimated Rating: 4.68
Movie ID: 3578, Estimated Rating: 4.64
Movie ID: 101, Estimated Rating: 4.62
Movie ID: 2268, Estimated Rating: 4.62


In [ ]:
from collections import defaultdict

# Function to count predictions for each user ID
def count_predictions(predictions):
    prediction_counts = defaultdict(int)
    for prediction in predictions:
        prediction_counts[prediction.uid] += 1
    return prediction_counts

# Generate predictions if not already done
# Ensure you have a predictions list from model.test(testset) or similar

# Count predictions for each user
prediction_counts = count_predictions(predictions)

# Print the number of predictions for each user ID
for user_id, count in prediction_counts.items():
    print(f"User ID {user_id} has {count} predictions.")

# You might want to print the counts for specific users, such as user 1 or 2
print(f"Number of predictions for user 1: {prediction_counts[1]}")
if 2 in prediction_counts:
    print(f"Number of predictions for user 2: {prediction_counts[2]}")
else:
    print("No predictions found for user 2.")


User ID 352 has 62 predictions.
User ID 89 has 99 predictions.
User ID 68 has 265 predictions.
User ID 140 has 121 predictions.
User ID 367 has 34 predictions.
User ID 263 has 45 predictions.
User ID 254 has 21 predictions.
User ID 91 has 98 predictions.
User ID 405 has 19 predictions.
User ID 227 has 18 predictions.
User ID 313 has 65 predictions.
User ID 204 has 15 predictions.
User ID 452 has 41 predictions.
User ID 301 has 26 predictions.
User ID 606 has 206 predictions.
User ID 608 has 190 predictions.
User ID 119 has 46 predictions.
User ID 307 has 195 predictions.
User ID 29 has 19 predictions.
User ID 200 has 59 predictions.
User ID 462 has 104 predictions.
User ID 469 has 99 predictions.
User ID 448 has 375 predictions.
User ID 527 has 33 predictions.
User ID 265 has 23 predictions.
User ID 125 has 70 predictions.
User ID 520 has 31 predictions.
User ID 474 has 396 predictions.
User ID 174 has 19 predictions.
User ID 134 has 6 predictions.
User ID 591 has 12 predictions.
User 

## Hyperparameter Tuning in KnnWithMeans

In [ ]:
# # Define a parameter grid to search
# param_grid = {
#     'k': [10,20, 30, 40, 50, 60],
#     'sim_options': {
#         'name': ['msd', 'cosine', 'pearson', 'pearson_baseline'],
#         'user_based': [True, False],
#         'min_support': [1, 5, 10],
#         'shrinkage': [0, 100]  # Only relevant for pearson_baseline
#     }
# }

# # Setup the grid search
# gs = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse', 'mae'], cv=2)

# # Perform grid search
# gs.fit(data)

# # Best RMSE score
# print('Best RMSE:', gs.best_score['rmse'])

# # Combination of parameters that gave the best RMSE score
# print('Best parameters:', gs.best_params['rmse'])

# Movie Recommendation : SVD

In [ ]:
# Initialize and train the SVD model
model = SVD()
start_time = time.time()
model.fit(trainset)
fit_time = time.time() - start_time
print(f"SVD training completed in {fit_time:.2f} seconds.")

SVD training completed in 1.68 seconds.


In [ ]:
# Make predictions on the test set
predictions = model.test(testset)

# Function to calculate precision and recall at k
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()

    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold)) for (est, true_r) in user_ratings[:k])

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return sum(prec for prec in precisions.values()) / len(precisions), sum(rec for rec in recalls.values()) / len(recalls)

# Evaluate precision, recall, and MSE
precision, recall = precision_recall_at_k(predictions)
mse = accuracy.mse(predictions, verbose=False)
rmse = np.sqrt(mse)
print(f"SVD: Precision={precision:.3f}, Recall={recall:.3f}, MSE={mse:.3f}, RMSE={rmse:.3f}")

SVD: Precision=0.736, Recall=0.513, MSE=0.764, RMSE=0.874


In [ ]:
# Function to generate recommendations
def generate_recommendations(user_id, predictions):
    user_predictions = [pred for pred in predictions if pred.uid == user_id]
    user_predictions.sort(key=lambda x: x.est, reverse=True)
    top_10_recommendations = user_predictions[:10]

    print(f"Some movies recommendations for User {user_id}:")
    for pred in top_10_recommendations:
        print(f"Movie ID: {pred.iid}, Estimated Rating: {pred.est:.2f}")

# Example usage
user_id = 1
generate_recommendations(user_id, predictions)

Some movies recommendations for User 1:
Movie ID: 1213, Estimated Rating: 5.00
Movie ID: 1617, Estimated Rating: 5.00
Movie ID: 1198, Estimated Rating: 4.98
Movie ID: 1219, Estimated Rating: 4.96
Movie ID: 923, Estimated Rating: 4.95
Movie ID: 1967, Estimated Rating: 4.92
Movie ID: 593, Estimated Rating: 4.92
Movie ID: 2959, Estimated Rating: 4.87
Movie ID: 110, Estimated Rating: 4.83
Movie ID: 1214, Estimated Rating: 4.79


In [ ]:
# Define the parameter grid for SVD
param_grid = {
    'n_factors': [50, 100, 150],  # Example values for n_factors
    'n_epochs': [5, 10, 20],      # Example values for n_epochs
    'lr_all': [0.002, 0.005, 0.007],  # Learning rates
    'reg_all': [0.02, 0.1, 0.4]   # Regularization terms
}

# Setup GridSearchCV
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

# Perform grid search
gs.fit(data)

# Best RMSE score
print('Best RMSE:', gs.best_score['rmse'])

# Combination of parameters that gave the best RMSE score
print('Best parameters:', gs.best_params['rmse'])


Best RMSE: 0.8720910660170033
Best parameters: {'n_factors': 100, 'n_epochs': 20, 'lr_all': 0.007, 'reg_all': 0.1}


# SVD++

In [ ]:
# Initialize and train the SVD++ model
model = SVDpp()
start_time = time.time()
model.fit(trainset)
fit_time = time.time() - start_time
print(f"SVD++ training completed in {fit_time:.2f} seconds.")

SVD++ training completed in 102.68 seconds.


In [ ]:
# Make predictions on the test set
predictions = model.test(testset)

# Function to calculate precision and recall at k
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()

    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold)) for (est, true_r) in user_ratings[:k])

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return sum(prec for prec in precisions.values()) / len(precisions), sum(rec for rec in recalls.values()) / len(recalls)

# Evaluate precision, recall, and MSE
precision, recall = precision_recall_at_k(predictions)
mse = accuracy.mse(predictions, verbose=False)
rmse = np.sqrt(mse)
print(f"SVD++: Precision={precision:.3f}, Recall={recall:.3f}, MSE={mse:.3f}, RMSE={rmse:.3f}")

SVD++: Precision=0.739, Recall=0.517, MSE=0.744, RMSE=0.862


In [ ]:
# Function to generate recommendations
def generate_recommendations(user_id, predictions):
    user_predictions = [pred for pred in predictions if pred.uid == user_id]
    user_predictions.sort(key=lambda x: x.est, reverse=True)
    top_10_recommendations = user_predictions[:10]

    print(f"Top 10 recommendations for User {user_id}:")
    for pred in top_10_recommendations:
        print(f"Movie ID: {pred.iid}, Estimated Rating: {pred.est:.2f}")

# Example usage
user_id = 1
generate_recommendations(user_id, predictions)

Top 10 recommendations for User 1:
Movie ID: 2959, Estimated Rating: 5.00
Movie ID: 1213, Estimated Rating: 5.00
Movie ID: 923, Estimated Rating: 5.00
Movie ID: 1219, Estimated Rating: 5.00
Movie ID: 1617, Estimated Rating: 5.00
Movie ID: 593, Estimated Rating: 5.00
Movie ID: 1198, Estimated Rating: 5.00
Movie ID: 1220, Estimated Rating: 4.89
Movie ID: 2078, Estimated Rating: 4.79
Movie ID: 2716, Estimated Rating: 4.77


In [ ]:
# Define the parameter grid for SVD++
param_grid = {
    'n_factors': [20, 50, 100],   # Example values for n_factors
    'n_epochs': [10, 20],         # Example values for n_epochs
    'lr_all': [0.007, 0.01],      # Learning rates
    'reg_all': [0.02, 0.1]        # Regularization terms
}

# Setup GridSearchCV
gs = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=3)

# Perform grid search
gs.fit(data)

# Best RMSE score
print('Best RMSE:', gs.best_score['rmse'])

# Combination of parameters that gave the best RMSE score
print('Best parameters:', gs.best_params['rmse'])

KeyboardInterrupt: 

# NMF

In [ ]:
# Initialize and train the NMF model
model = NMF()
start_time = time.time()
model.fit(trainset)
fit_time = time.time() - start_time
print(f"NMF training completed in {fit_time:.2f} seconds.")

NMF training completed in 2.87 seconds.


In [ ]:
# Make predictions on the test set
predictions = model.test(testset)

# Function to calculate precision and recall at k
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()

    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold)) for (est, true_r) in user_ratings[:k])

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return sum(prec for prec in precisions.values()) / len(precisions), sum(rec for rec in recalls.values()) / len(recalls)

# Evaluate precision, recall, and MSE
precision, recall = precision_recall_at_k(predictions)
mse = accuracy.mse(predictions, verbose=False)
rmse = np.sqrt(mse)
print(f"NMF: Precision={precision:.3f}, Recall={recall:.3f}, MSE={mse:.3f}, RMSE={rmse:.3f}")

NMF: Precision=0.724, Recall=0.484, MSE=0.866, RMSE=0.931


In [ ]:
# Function to generate recommendations
def generate_recommendations(user_id, predictions):
    user_predictions = [pred for pred in predictions if pred.uid == user_id]
    user_predictions.sort(key=lambda x: x.est, reverse=True)
    top_10_recommendations = user_predictions[:10]

    print(f"Some movies recommendations for User {user_id}:")
    for pred in top_10_recommendations:
        print(f"Movie ID: {pred.iid}, Estimated Rating: {pred.est:.2f}")

# Example usage
user_id = 1
generate_recommendations(user_id, predictions)

Some movies recommendations for User 1:
Movie ID: 2648, Estimated Rating: 5.00
Movie ID: 1213, Estimated Rating: 5.00
Movie ID: 923, Estimated Rating: 5.00
Movie ID: 1219, Estimated Rating: 5.00
Movie ID: 3740, Estimated Rating: 5.00
Movie ID: 1617, Estimated Rating: 5.00
Movie ID: 2944, Estimated Rating: 5.00
Movie ID: 1348, Estimated Rating: 4.94
Movie ID: 2654, Estimated Rating: 4.77
Movie ID: 1967, Estimated Rating: 4.74


# Baselineonly

In [ ]:
# Initialize and train the BaselineOnly model
bsl_options = {'method': 'als', 'n_epochs': 5, 'reg_u': 12, 'reg_i': 5}
model = BaselineOnly(bsl_options=bsl_options)
start_time = time.time()
model.fit(trainset)
fit_time = time.time() - start_time
print(f"BaselineOnly training completed in {fit_time:.2f} seconds.")

Estimating biases using als...
BaselineOnly training completed in 0.11 seconds.


In [ ]:
# Make predictions on the test set
predictions = model.test(testset)

# Function to calculate precision and recall at k
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()

    for uid, user_ratings in user_est_true.items():
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold)) for (est, true_r) in user_ratings[:k])

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return sum(prec for prec in precisions.values()) / len(precisions), sum(rec for rec in recalls.values()) / len(recalls)

# Evaluate precision, recall, and MSE
precision, recall = precision_recall_at_k(predictions)
mse = accuracy.mse(predictions, verbose=False)
rmse = np.sqrt(mse)
print(f"BaselineOnly: Precision={precision:.3f}, Recall={recall:.3f}, MSE={mse:.3f}, RMSE={rmse:.3f}")

BaselineOnly: Precision=0.738, Recall=0.524, MSE=0.757, RMSE=0.870


In [ ]:
# Function to generate recommendations
def generate_recommendations(user_id, predictions):
    user_predictions = [pred for pred in predictions if pred.uid == user_id]
    user_predictions.sort(key=lambda x: x.est, reverse=True)
    top_10_recommendations = user_predictions[:10]

    print(f"Some Movies recommendations for User {user_id}:")
    for pred in top_10_recommendations:
        print(f"Movie ID: {pred.iid}, Estimated Rating: {pred.est:.2f}")

# Example usage
user_id = 1
generate_recommendations(user_id, predictions)

Some Movies recommendations for User 1:
Movie ID: 608, Estimated Rating: 4.83
Movie ID: 1219, Estimated Rating: 4.83
Movie ID: 923, Estimated Rating: 4.81
Movie ID: 1291, Estimated Rating: 4.78
Movie ID: 110, Estimated Rating: 4.75
Movie ID: 1206, Estimated Rating: 4.74
Movie ID: 1265, Estimated Rating: 4.70
Movie ID: 2761, Estimated Rating: 4.65
Movie ID: 1090, Estimated Rating: 4.65
Movie ID: 1073, Estimated Rating: 4.63
